The life of a value in Mojo begins when a variable is initialized and continues
up until the value is last used, at which point Mojo destroys it. This page
describes how every value in Mojo is created, copied, and moved. (The next
page describes [how values are
destroyed](/mojo/manual/lifecycle/death.html).)

All data types in Mojo—including basic types in the standard library such as
[`Bool`](/mojo/stdlib/builtin/bool.html#bool),
[`Int`](/mojo/stdlib/builtin/int.html#int), and
[`String`](/mojo/stdlib/builtin/string.html#string), up to complex types such
as [`SIMD`](/mojo/stdlib/builtin/simd.html#simd) and
[`object`](/mojo/stdlib/builtin/object.html#object)—are all defined as a
[struct](/mojo/manual/structs.html). This means the creation and
destruction of any piece of data follows the same lifecycle rules, and you can
define your own data types that work exactly the same way.

Mojo structs don't get any default lifecycle methods, such as a
constructor, copy constructor, or move constructor. That means you can create
a struct without a constructor, but then you can't instantiate it, and it
would be useful only as a sort of namespace for static methods. For example:

In [1]:
struct NoInstances:
    var state: Int

    @staticmethod
    fn print_hello():
        print("Hello world!")

Without a constructor, this cannot be instantiated, so it has no lifecycle. The
`state` field is also useless because it cannot be initialized (Mojo structs do
not support default field values—you must initialize them in a constructor).

So the only thing you can do is call on the static method:

In [2]:
NoInstances.print_hello()

Hello world!


## Constructor

To create an instance of a Mojo type, it needs the `__init__()` constructor
method. The main responsibility of the constructor is to initialize all fields.
For example:

In [3]:
struct MyPet:
    var name: String
    var age: Int

    fn __init__(inout self, name: String, age: Int):
        self.name = name
        self.age = age

Now we can create an instance:

In [4]:
let mine = MyPet("Loki", 4)

An instance of `MyPet` can also be
[borrowed](/mojo/manual/values/ownership.html#immutable-arguments-borrowed)
and destroyed, but it currently can't be copied or moved.

We believe this is a good default starting point, because there are no built-in
lifecycle events and no surprise behaviors. You—the type author—must
explicitly decide whether and how the type can be copied or moved, by
implementing the copy and move constructors.

:::{.callout-note}

**Note:** Mojo does not require a destructor to destroy an object. As long as
all fields in the struct are destructible (every type in the standard library
is destructible, except for
[pointers](/mojo/stdlib/memory/unsafe.html)), then Mojo knows how to destroy
the type when its lifetime ends. We'll discuss that more in [Death of a
value](/mojo/manual/lifecycle/death.html).

:::

### Overloading the constructor

Like any other function/method, you can
[overload](/mojo/manual/functions.html#overloaded-functions) the
`__init__()` constructor to initialize the object with different arguments. For
example, you might want a default constructor that sets some default values and
takes no arguments, and then additional constructors that accept more arguments.

Just beware that, in order to modify any fields, each constructor must
declare the `self` argument with the [`inout`
convention](/mojo/manual/values/ownership.html#mutable-arguments-inout). If you
want to call one constructor from another, you simply call upon that
constructor as you would externally (you don't need to pass `self`).

For example, here's how you can delegate work from an overloaded constructor:

In [5]:
struct MyPet:
    var name: String
    var age: Int

    fn __init__(inout self):
        self.name = ""
        self.age = 0

    fn __init__(inout self, name: String):
        self = MyPet()
        self.name = name

### Field initialization

Notice in the previous example that, by the end of each constructor, all fields
must be initialized. That's the only requirement in the constructor.

In fact, the `__init__()` constructor is smart enough to treat the `self`
object as fully initialized even before the constructor is finished, as long
as all fields are initialized. For example, this constructor can pass around
`self` as soon as all fields are initialized:

In [6]:
fn use(arg: MyPet):
    pass

struct MyPet:
    var name: String
    var age: Int

    fn __init__(inout self, name: String, age: Int, cond: Bool):
        self.name = name
        if cond:
            self.age = age
            use(self)  # Safe to use immediately!

        self.age = age
        use(self)  # Safe to use immediately!

## Copy constructor

Because Mojo is built upon [value
semantics](/mojo/manual/values/value-semantics.html), Mojo makes a copy of any
value given to the assignment operator (`=`), by default. Also recall that
arguments sent to a [`def`
function](/mojo/manual/functions.html#def-functions) are copied by
default. However, copying a value is possible only if the type implements the
`__copyinit__()` method.

For example, the `MyPet` type above does not have a copy constructor,
so this code fails to compile:

```mojo
let mine = MyPet("Loki", 4)
let yours = mine  # This requires a copy, but MyPet has no copy constructor
```

To make it work, we need to add the copy constructor, like
this:

In [7]:
struct MyPet:
    var name: String
    var age: Int

    fn __init__(inout self, name: String, age: Int):
        self.name = name
        self.age = age

    fn __copyinit__(inout self, existing: Self):
        self.name = existing.name
        self.age = existing.age

:::{.callout-note}

**Note:** `Self` (capital "S") is an alias for the current type name
(`MyPet`, in this example). Using this alias is a best practice to avoid any
mistakes when referring to the current struct name.

Also, notice that the `existing` argument in `__copyinit__()` is immutable
because the default [argument
convention](/mojo/manual/values/ownership.html#argument-conventions) in an `fn`
function is `borrowed`—this is a good thing because this function should not
modify the contents of the value being copied.

:::

Now this code works to make a copy:

In [8]:
let mine = MyPet("Loki", 4)
let yours = mine

What makes Mojo's copy behavior different, compared to other languages, is that
`__copyinit__()` is designed to perform a deep copy of all fields in the type
(as per [value semantics](/mojo/manual/values/value-semantics.html)). That is,
it copies heap-allocated values, rather than just copying the pointer.

However, the Mojo compiler doesn't enforce this, so it's the type author's
responsibility to implement `__copyinit__()` with value semantics. For example,
here's a new `HeapArray` type that performs a deep copy in the copy constructor:

In [9]:
struct HeapArray:
    var data: Pointer[Int]
    var size: Int
    var cap: Int

    fn __init__(inout self, size: Int, val: Int):
        self.size = size
        self.cap = size * 2
        self.data = Pointer[Int].alloc(self.cap)
        for i in range(self.size):
            self.data.store(i, val)

    fn __copyinit__(inout self, existing: Self):
        # Deep-copy the existing value
        self.size = existing.size
        self.cap = existing.cap
        self.data = Pointer[Int].alloc(self.size)
        for i in range(self.size):
            self.data.store(i, existing.data.load(i))
        # The lifetime of `existing` continues unchanged

    fn __del__(owned self):
        # We must free the heap-allocated data, but
        # Mojo knows how to destroy the other fields
        self.data.free()

    fn append(inout self, val: Int):
        # Update the array for demo purposes
        if self.size < self.cap:
            self.data.store(self.size, val)
            self.size += 1
        else:
            print("Out of bounds")

    fn dump(self):
        # Print the array contents for demo purposes
        print_no_newline("[")
        for i in range(self.size):
            if i > 0:
                print_no_newline(", ")
            print_no_newline(self.data.load(i))
        print("]")

Notice that `__copyinit__()` does not copy the `Pointer` value (doing so would
make the copied value refer to the same `data` memory address as the original
value, which is a shallow copy). Instead, we initialize a new `Pointer` to
allocate a new block of memory, and then copy over all the heap-allocated
values (this is a deep copy).

Thus, when we copy an instance of `HeapArray`, each copy has its own value on
the heap, so changes to one value do not affect the other, as shown here:

In [10]:
fn copies():
    let a = HeapArray(2, 1)
    var b = a    # Calls the copy constructor
    a.dump()     # Prints [1, 1]
    b.dump()     # Prints [1, 1]

    b.append(2)  # Changes the copied data
    b.dump()     # Prints [1, 1, 2]
    a.dump()     # Prints [1, 1] (the original did not change)

:::{.callout-note}

**Note:** In `HeapArray`, we must use the `__del__()` destructor to free the
heap-allocated data when the `HeapArray` lifetime ends, but Mojo automatically
destroys all other fields when their respective lifetimes end. We'll discuss
this destructor more in [Death of a value](/mojo/manual/lifecycle/death.html).

:::

If your type doesn't use any pointers for heap-allocated data, then writing the
constructor and copy constructor is all boilerplate code that you shouldn't
have to write. So, if all your fields use destructible data types (such as
`Int`, `Bool`, `String`, etc.), you can just add the [`@value`
decorator](/mojo/manual/decorators/value.html) to your struct definition and
Mojo will synthesize the `__init__()`, `__copyinit__()`, and `__moveinit__()`
methods.

## Move constructors

Although copying values provides predictable behavior that matches Mojo's
[value semantics](/mojo/manual/values/value-semantics.html), copying some data
types can be a significant hit on performance. If you're familiar with
reference semantics, then the solution here might seem clear: instead of making
a copy when passing a value, share the value as a reference. And if the
original variable is no longer needed, nullify the original to avoid any
double-free or use-after-free errors. That's generally known as a move
operation: the memory block holding the data remains the same (the memory does
not actually move), but the pointer to that memory moves to a new variable in
the call stack.

However, "move" can mean something different depending on whether you're more
experienced with C++ or Rust, which each offer their own version of move
semantics that avoid copying heap-allocated values. Each approach has its
benefits, so Mojo supports them both with a separate constructor:

- `__moveinit__()` is invoked to perform a "consuming" move, which
  fully transfers ownership of a value without copying heap-allocated data.
  Mojo ends the lifetime of the original value (the variable becomes invalid)
  to avoid double-free and use-after-free errors. (This is a Rust-style move;
  also called a "destructive move.")

- `__takeinit__()` is invoked to perform a "taking" move, which is
  similar in that it moves ownership of a value without copying heap-allocated
  data, except it does _not_ end the lifetime of the original value. As the type
  author, it's your responsibility to update the original variable with a "null
  state." (This is a C++-style move; also called a "stealing move" or a
  "non-destructive move.")

The difference between these two—especially how they are each invoked—might be
a bit confusing now, but it should become more clear in the following sections.
Also, it's safe to say that you generally don't need to worry about using
`__takeinit__()` unless you already know that this is necessary for your
problem, as it is generally useful only in rare circumstances.

:::{.callout-note}

**Note:** A move constructor is **not required** to transfer ownership of a
value. Unlike in Rust, transferring ownership is not always a move operation;
the move constructors are only part of the implementation for how Mojo
transfers ownership of a value. You can learn more in the section about
[ownership
transfer](/mojo/manual/values/ownership.html#transfer-arguments-owned-and).

:::

### Consuming-move constructor

The `__moveinit__()` method performs a consuming-move: it transfers ownership
of a value from one variable to another when the original variable's lifetime
ends (also called a "destructive move").

For most situations, this is the behavior you want when you [transfer
ownership](/mojo/manual/values/ownership.html#transfer-arguments-owned-and),
because Mojo invokes `__moveinit__()` to transfer ownership only when Mojo can
guarantee that the lifetime of the original value is ending (usually indicated
by the `^` transfer operator). (However, Mojo can transfer ownership in some
situations even without this constructor.)

When a consuming-move occurs, Mojo immediately invalidates the original
variable, preventing any access to it and disabling its destructor. Invalidating
the original variable is important to avoid memory errors on heap-allocated
data, such as use-after-free and double-free errors.

Here's how to add the consuming-move constructor to the `HeapArray` example:

In [11]:
struct HeapArray:
    var data: Pointer[Int]
    var size: Int

    fn __init__(inout self, size: Int, val: Int):
        self.size = size
        self.data = Pointer[Int].alloc(self.size)
        for i in range(self.size):
            self.data.store(i, val)

    fn __copyinit__(inout self, existing: Self):
        # Deep-copy the existing value
        self.size = existing.size
        self.data = Pointer[Int].alloc(self.size)
        for i in range(self.size):
            self.data.store(i, existing.data.load(i))

    fn __moveinit__(inout self, owned existing: Self):
        print("move")
        # Shallow copy the existing value
        self.size = existing.size
        self.data = existing.data
        # Then the lifetime of `existing` ends here, but
        # Mojo does NOT call its destructor

    fn __del__(owned self):
        self.data.free()

    fn dump(self):
        print_no_newline("[")
        for i in range(self.size):
            if i > 0:
                print_no_newline(", ")
            print_no_newline(self.data.load(i))
        print("]")

The critical feature of `__moveinit__()` is that it takes the incoming value as
`owned`, meaning this method gets unique ownership of the value. Moreover,
because this is a dunder method that Mojo calls only when performing a
consuming-move (during ownership transfer), the `existing` argument is
guaranteed to be a mutable reference to the original value, _not a copy_
(unlike other methods that may declare an argument as `owned`, but might
receive the value as a copy if the method is called without the [`^` transfer
operator](/mojo/manual/values/ownership.html#transfer-arguments-owned-and)).
That is, Mojo calls this move constructor _only_ when the original variable's
lifetime actually ends at the point of transfer.

Here's an example showing how to invoke the move constructor for `HeapArray`:

In [12]:
fn moves():
    let a = HeapArray(3, 1)

    a.dump()   # Prints [1, 1, 1]

    let b = a^ # Prints "move"; the lifetime of `a` ends here

    b.dump()   # Prints [1, 1, 1]
    #a.dump()  # ERROR: use of uninitialized value 'a'

Notice that `__moveinit__()` performs a shallow copy of the
existing field values (it copies the pointer, instead of allocating new memory
on the heap), which is what makes it useful for types with heap-allocated
values that are expensive to copy.

To go further and ensure your type can never be copied, you can make it
"move-only" by implementing `__moveinit__()` and _excluding_ `__copyinit__()`.
A move-only type can be passed to other variables and passed into functions
with any argument convention (`borrowed`, `inout`, and `owned`)—the only catch
is that you must use the `^` transfer operator to end the lifetime of a
move-only type when assigning it to a new variable or when passing it as an
`owned` argument.

:::{.callout-note}

**Note:** For types without heap-allocated fields, you get no real benefit from
the move constructor. Making copies of simple data types on the stack, like
integers, floats, and booleans, is very cheap. Yet, if you allow your type to
be copied, then there's generally no reason to disallow moves, so you can
synthesize both constructors by adding the [`@value`
decorator](/mojo/manual/decorators/value.html).

:::

### Taking-move constructor

Although `__moveinit__()` provides a mechanism to provide predictable ownership
transfers for your types, it enforces move semantics that are problematic in
(rare) situations where you don't want the lifetime of the original value to
end. In such situations, you might prefer to implement `__takeinit__()`, which
performs a "taking move" (also known as a "stealing move" or "non-destructive
move").

The `__takeinit__()` constructor has a similar effect as the `__moveinit__()`
constructor, but it does not end the lifetime of the original value. This is
useful for situations in which it's necessary that the original variable remain
valid (but in a "null state"), so that its destructor is still allowed to run
(but only upon a nullified value to avoid a double-free).

Generally, you'll know it if you need to use this; if `__moveinit__()` makes
sense and that behavior doesn't worry you, then you don't need
`__takeinit__()`.

:::{.callout-caution}

**Caution:** Currently, this documentation about `__takeinit__()` describes our
design intentions, and the lifetime behaviors required to invoke
`__takeinit__()` when a value is passed to a function that takes ownership is
not fully implemented. There are several TODOs we must finish before this is
fully functional, such as the ability for code to [return
references](/mojo/roadmap.html#ownership-and-lifetimes).

:::

For example, let's say you create an array of `HeapArray` values. If you
transfer ownership of just one item in that array using `__moveinit__()`, you
might encounter problems using that array later, because the array will contain
an invalid variable that cannot be read or destroyed. To solve for this
usecase, you might want to implement `__takeinit__()` instead of
`__moveinit__()` so that all array items remain valid.

However, because this move operation leaves the original value valid, it allows
for potential double-free and use-after-free errors. So, it's very important
that you hollow-out the original value in your `__takeinit__()` implementation,
by replacing the original value contents with some type of "null state" or
sentinel value that tells the destructor to treat it differently.

For example, here's how we can implement `HeapArray` to allow a taking-move:

In [13]:
struct HeapArray:
    var data: Pointer[Int]
    var size: Int

    fn __init__(inout self, size: Int, val: Int):
        self.size = size
        self.data = Pointer[Int].alloc(self.size)
        for i in range(self.size):
            self.data.store(i, val)

    fn __takeinit__(inout self, inout existing: Self):
        # Shallow-copy the existing value
        self.size = existing.size
        self.data = existing.data
        # The lifetime for `existing` does not end here, so we
        # must update its value to no longer refer to moved data,
        # in order to avoid double-free and use-after-free errors,
        # but in a way that still allows the destructor to run.
        existing.size = 0
        existing.data = Pointer[Int].get_null()

    fn __del__(owned self):
        # Free the data only if the Pointer is not null
        if (self.data):
            self.data.free()

    fn dump(self):
        print_no_newline("[")
        for i in range(self.size):
            if i > 0:
                print_no_newline(", ")
            print_no_newline(self.data.load(i))
        print("]")

This technique has tradeoffs and is not the best for every type. However, we
believe that you, the library authors, understand your domain problems better
than we, the language designers do. So we prefer to provide full control over
these behaviors. As such you can choose to (but don’t have to) make your types
participate in this taking-move constructor.

:::{.callout-note}

**Note:** Because `__takeinit__()` uses `inout` for the `existing` argument,
the value you want to move with the taking-move operation must be mutable.

:::

## Simple value types {#value-decorator}

Because copy and move constructors are opt-in, Mojo provides great control for
exotic usecases (such as for atomic values that should never be copied or
moved), but most structs are simple aggregations of other types that should be
easily copied and moved, and we don't want to write a lot of boilerplate
constructors for those simple value types.

To solve this, Mojo provides the [`@value`
decorator](/mojo/manual/decorators/value.html), which synthesizes the
boilerplate code for the `__init__()`, `__copyinit__()`, and `__moveinit__()`
methods.

For example, consider a simple struct like this:

In [14]:
@value
struct MyPet:
    var name: String
    var age: Int

Mojo sees the `@value` decorator and notices that you don't have a member-wise
initializer (a constructor with arguments for each field), a copy constructor,
or a move constructor, so it synthesizes them for you. The result is as if you
had actually written this:

In [15]:
struct MyPet:
    var name: String
    var age: Int

    fn __init__(inout self, owned name: String, age: Int):
        self.name = name^
        self.age = age

    fn __copyinit__(inout self, existing: Self):
        self.name = existing.name
        self.age = existing.age

    fn __moveinit__(inout self, owned existing: Self):
        self.name = existing.name^
        self.age = existing.age

Mojo synthesizes each lifecycle method only when it doesn't exist, so
you can use `@value` and still define your own versions to override the default
behavior. For example, it is fairly common to use the default member-wise and
move constructor, but create a custom copy constructor.

Something you can see in this code that we didn't mention yet is that the
`__init__()` method takes all arguments as `owned`, because the constructor
must take ownership to store each value. This is a useful micro-optimization
and enables the use of move-only types. Trivial types like `Int` are also
passed as `owned`, but because ownership doesn't mean anything for integers, we
can elide that declaration and the transfer operator (`^`) for simplicity. The
transfer operator is also just a formality in this case, because, even if it's
not used with `self.name = name^`, the Mojo compiler will notice that `name` is
last used here and convert this assignment into a move, instead of a
copy+delete.

:::{.callout-note}

**Note:** If your type contains any move-only fields, Mojo will not generate
the copy constructor because it cannot copy those fields. Further, the `@value`
decorator won't work at all if any of your members are neither copyable nor
movable. For example, if you have something like `Atomic` in your struct, then
it probably isn't a true value type, and you don't want the copy/move
constructors anyway.

Also notice that the `MyPet` struct above doesn't include the `__del__()`
destructor (the `@value` decorator does not synthesize this), because Mojo
doesn't need it to destroy fields, as discussed in [Death of a
value](/mojo/manual/lifecycle/death.html)

:::

## Trivial types

So far, we've talked about values that live in memory, which means they have an
identity (an address) that can be passed around among functions (passed "by
reference"). This is great for most types, and it's a safe default for large
objects with expensive copy operations. However, it's inefficient for tiny
things like a single integer or floating point number. We call these types
"trivial" because they are just "bags of bits" that should be copied, moved,
and destroyed without invoking any custom lifecycle methods.

Trivial types are the most common types that surround us, and from a language
perspective, Mojo doesn’t need special support for these written in a struct.
Usually, these values are so tiny that they should be passed around in CPU
registers, not indirectly through memory.

As such, Mojo provides a struct decorator to declare these types of values:
`@register_passable("trivial")`. This decorator tells Mojo that the type should
be copyable and movable but that it has no user-defined logic (no lifecycle
methods) for doing this. It also tells Mojo to pass the value in CPU registers
whenever possible, which has clear performance benefits.

You'll see this decorator on types like `Int` in the standard library:

```mojo
@register_passable("trivial")
struct Int:
    var value: __mlir_type.index

    fn __init__(value: __mlir_type.index) -> Int:
        return Self {value: value}
    ...
```

We expect to use this decorator pervasively on Mojo standard library types, but
it is safe to ignore for general application-level code.

For more information, see the [`@register_passable`
documentation](/mojo/manual/decorators/register-passable.html).

:::{.callout-note}

TODO: This decorator is due for reconsideration.  Lack of custom
copy/move/destroy logic and "passability in a register" are orthogonal concerns
and should be split.  This former logic should be subsumed into a more general
`@value("trivial")` decorator, which is orthogonal from `@register_passable`.

:::